In [44]:
import pandas as pd
import os 
import string

In [80]:
translation_file = "message_generation_sample.csv"
source_data_path = "data/negotiate"
dest_data_path = "data/negotiate_prepend"
file_name = "val.txt"
prepend_mode = True

## Read Translation File

In [81]:
df_trans = pd.read_csv(translation_file, index_col =0)
df_trans['sentences'] = df_trans['sentences'].astype(str)
df_trans['Predicted'] = df_trans['Predicted'].astype(str)
translate = {}
for ind, row in df_trans.iterrows():
    translate[row['sentences'].strip()] = row['Predicted'].strip()

In [82]:
def translate_dialog(dialog, translate_dict):
    translated_dialog = ''
    messages = dialog.split('<eos>')
    for m in messages:
        m_split = m.split(":", 1)
        if m_split[1].strip() == '<selection>':
            translated_dialog = translated_dialog + m
        else:
            if m_split[1].strip() in translate_dict.keys():
                translation = translate_dict[m_split[1].strip()]
            else:
                translation = '_unknown'
            if not prepend_mode:
                translated_dialog = translated_dialog + m_split[0] + f': {translation} <eos> '
            else:
                translated_dialog = translated_dialog + m_split[0] + f': {translation} - {m_split[1]} <eos> '
    return translated_dialog
            

## Read Data File

In [83]:
lines = []
with open(os.path.join(source_data_path, file_name), 'r') as f:
    for line in f:
        lines.append(line.strip())



In [84]:
sep1 = '<dialogue>'
sep2 = '</dialogue>'

new_lines = []
for line in lines:
    l_split1 = line.split(sep1)
    start = l_split1[0] + sep1
    l_split2 = l_split1[1].split(sep2)
    dialogue = l_split2[0]
    end = sep2 + l_split2[1] +'\n'
    new_dialog = translate_dialog(dialogue, translate)
    new_lines.append(start + new_dialog + end)

In [85]:
if not os.path.exists(dest_data_path):
    os.mkdir(dest_data_path)
out_file = os.path.join(dest_data_path, file_name)
with open(out_file, 'w') as f:
    f.writelines(new_lines)
    

In [86]:
new_lines

['<input> 1 6 3 0 2 2 </input> <dialogue> YOU: _need( _books 1 _balls all ) _offer( _hats all ) -  you can have all the hats if i get the book and basketballs .  <eos>  THEM: <selection> </dialogue> <output> <disagree> <disagree> <disagree> <disagree> <disagree> <disagree> </output> <partner_input> 1 2 3 2 2 1 </partner_input>\n',
 '<input> 3 3 3 0 1 1 </input> <dialogue> YOU: _need( _books all ) _offer( _hats all _balls 1 ) -  im a reader , so id like the books . . . . you may have the hats and ball  <eos>  THEM: _need( _books 2 _hats all ) -  let me have two books and the hats  <eos>  YOU: _need( _books all ) -  its a trilogy so i really need to hold on to all the books  <eos>  THEM: _nodeal -  cant do it  <eos>  YOU: _counteroffer _need( _books 2 _balls 1 ) -  ok , well best i can do is 2 books and the ball then . . . anything less and i cant make a deal  <eos>  THEM: _need( _books 1 _hats all ) -  so the hats and a book for me ?  <eos>  YOU: _deal -  yes  <eos>  THEM: <selection> <